In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pandas.read_csv("x.csv")
dataframe.Hospital_ID= dataframe.Hospital_ID.astype(str)
dataframe.District_ID= dataframe.District_ID.astype(str)
dataframe.Instrument_ID= dataframe.Instrument_ID.astype(str)
Y = dataframe['Binary'].values
del dataframe['Binary']
dataset = dataframe.values
# split into input (X) and output (Y) variables
X= dataset.astype(float)

In [4]:
dataframe_test = pandas.read_csv("Solution.csv")
Hospital_ID = []
District_ID = []
Instrument_ID = []
for i in range(len(dataframe_test)):
    Hospital_ID.append(int(filter(str.isdigit, dataframe_test['Hospital_ID'].values[i])))
    District_ID.append(int(filter(str.isdigit, dataframe_test['District_ID'].values[i])))
    Instrument_ID.append(int(filter(str.isdigit, dataframe_test['Instrument_ID'].values[i])))
dataframe_test['Hospital_ID'] =  Hospital_ID  
dataframe_test['District_ID'] =  District_ID 
dataframe_test['Instrument_ID'] =  Instrument_ID
dataframe_test.Hospital_ID= dataframe_test.Hospital_ID.astype(str)
dataframe_test.District_ID= dataframe_test.District_ID.astype(str)
dataframe_test.Instrument_ID= dataframe_test.Instrument_ID.astype(str)
del dataframe_test['Revenue']
del dataframe_test['Buy_or_not']
dataset_test = dataframe_test.values
# split into input (X) and output (Y) variables
X_test= dataset.astype(float)

In [5]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=3, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=1, batch_size=5, verbose=0)

In [ ]:
estimator.fit(X,Y)

In [28]:
pred = estimator.predict(X_test)
df_sol = pandas.read_csv('Solution.csv')
df_sol['Buy_or_not'] = pred
from pandas import ExcelWriter
writer = ExcelWriter('output.xlsx')
df_sol.to_excel(writer,'Sheet1')
writer.save()

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [42]:
# (h,d) pair -> employees
hd_to_employees = {}
for index, row in hospitals.iterrows():
    hid = get_id(row['Hospital_ID'], 'Hospital')
    did = get_id(row['District_ID'], 'District')
    employees = row['Hospital_employees']
    hd_to_employees[(hid, did)] = employees

In [43]:
N = len(revenue)
hids = np.zeros((N,1), dtype='uint')
dids = np.zeros((N,1), dtype='uint')
iids = np.zeros((N,1), dtype='uint')
emps = np.zeros((N,1))
revs = np.zeros((N,1)) #to predict

In [44]:
for i, row in revenue.iterrows():
    hids[i] = get_id(row['Hospital_ID'], 'Hospital')
    dids[i] = get_id(row['District_ID'], 'District')
    iids[i] = get_id(row['Instrument_ID'], 'Instrument')
    emps[i] = hd_to_employees.get((hids[i][0], dids[i][0]),-1)
    revs[i] = row['Year Total']

In [45]:
revs_mean = revs.mean()
revs_std = revs.std()
revs = (revs - revs_mean)/(revs_std)

In [41]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, merge, Embedding, Reshape
from keras.optimizers import SGD
from keras.models import Model


def get_id(id_str, prop):
    try:
        return int(id_str[len(prop):])
    except ValueError:
        return 0


hospitals = pd.read_csv('HospitalProfiling.csv')
revenue = pd.read_csv('HospitalRevenue.csv')

In [48]:
emb_len = 20

# Keras model
hid_node = Input(shape=(1,), dtype='int32')
did_node = Input(shape=(1,), dtype='int32')
iid_node = Input(shape=(1,), dtype='int32')
emp_node = Input(shape=(1,))

hid_emb = Reshape((emb_len,))(Embedding(hids.max()+1, emb_len)(hid_node))
did_emb = Reshape((emb_len,))(Embedding(dids.max()+1, emb_len)(did_node))
iid_emb = Reshape((emb_len,))(Embedding(iids.max()+1, emb_len)(iid_node))

merged_ftrs = merge([hid_emb, did_emb, iid_emb], 'concat')
pred = Dense(1)(Dense(64, activation='relu')(merged_ftrs))

model = Model(input=[hid_node, did_node, iid_node], 
                output=pred)

model.compile(optimizer=SGD(lr=1e-2), loss='mae')

model.fit([hids, dids, iids], revs, validation_split=0.3, nb_epoch=1)

Train on 31047 samples, validate on 13306 samples
Epoch 1/1
31047/31047 [==============================] - 1430s - loss: 0.1555 - val_loss: 0.1422


In [49]:
Solution = pd.read_csv('Solution.csv')
N = len(Solution)
hi = np.zeros((N,1), dtype='uint')
di = np.zeros((N,1), dtype='uint')
ii = np.zeros((N,1), dtype='uint')
for i, row in Solution.iterrows():
    hi[i] = get_id(row['Hospital_ID'], 'Hospital')
    di[i] = get_id(row['District_ID'], 'District')
    ii[i] = get_id(row['Instrument_ID'], 'Instrument')

In [50]:
import pandas
import numpy as np
df_hp = pandas.read_csv('HospitalProfiling.csv') 
df_hr = pandas.read_csv('HospitalRevenue.csv') 
df_pr = pandas.read_csv('ProjectedRevenue.csv') 
df_s = pandas.read_csv('Solution.csv')
df_x = pandas.read_csv('x.csv')
df_pr = df_pr[df_pr.District_ID != 'No District Available']
df_hr = df_hr[df_hr.District_ID != 'No District Available']

In [ ]:
Hospital_ID = []
District_ID = []
for i in range(len(df_hp)):
    Hospital_ID.append(int(filter(str.isdigit, df_hp['Hospital_ID'].values[i])))
    District_ID.append(int(filter(str.isdigit, df_hp['District_ID'].values[i])))
df_hp['Hospital_ID'] =  Hospital_ID  
df_hp['District_ID'] =  District_ID 

Hospital_ID = []
District_ID = []
Region_ID = []
Instrument_ID = []
for i in range(len(df_hr)):
    Hospital_ID.append(int(filter(str.isdigit, df_hr['Hospital_ID'].values[i])))
    District_ID.append(int(filter(str.isdigit, df_hr['District_ID'].values[i])))
    Region_ID.append(int(filter(str.isdigit, df_hr['Region_ID'].values[i])))
    Instrument_ID.append(int(filter(str.isdigit, df_hr['Instrument_ID'].values[i])))
df_hr['Hospital_ID'] =  Hospital_ID  
df_hr['District_ID'] =  District_ID 
df_hr['Region_ID'] =  Region_ID
df_hr['Instrument_ID'] =  Instrument_ID

Hospital_ID = []
District_ID = []
Instrument_ID = []
for i in range(len(df_pr)):
    Hospital_ID.append(int(filter(str.isdigit, df_pr['Hospital_ID'].values[i])))
    District_ID.append(int(filter(str.isdigit, df_pr['District_ID'].values[i])))
    Instrument_ID.append(int(filter(str.isdigit, df_pr['Instrument_ID'].values[i])))
df_pr['Hospital_ID'] =  Hospital_ID  
df_pr['District_ID'] =  District_ID 
df_pr['Instrument_ID'] =  Instrument_ID


Hospital_ID = []
District_ID = []
Instrument_ID = []
for i in range(len(df_s)):
    Hospital_ID.append(int(filter(str.isdigit, df_s['Hospital_ID'].values[i])))
    District_ID.append(int(filter(str.isdigit, df_s['District_ID'].values[i])))
    Instrument_ID.append(int(filter(str.isdigit, df_s['Instrument_ID'].values[i])))
df_s['Hospital_ID'] =  Hospital_ID  
df_s['District_ID'] =  District_ID 
df_s['Instrument_ID'] =  Instrument_ID

In [ ]:
#ENCODING
Y = df_x['Binary'].values
del df_x['Binary']
X = df_x.to_dict('records')

In [ ]:
from sklearn.feature_extraction import DictVectorizer 
enc = DictVectorizer()
X_test = df_s.to_dict('records')
X_train =enc.fit_transform(X)

In [ ]:

split = 0.8
X_encoded_total =enc.fit_transform(X)
X_encoded_train =X_encoded_total[:split*len(X)]
X_encoded_test =X_encoded_total[split*len(X):]
Y_train  = Y[:split*len(Y)]
Y_test = Y[split*len(Y):]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor,BaggingRegressor,RandomForestRegressor
reg1 = RandomForestRegressor(n_jobs=-1)
reg1.fit(X_encoded_train.toarray(),Y_train)

In [ ]:
pred = reg1.(X_encoded_tes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#OUR DATA
X = np.arange(1000)
Y = Y_test[:1000]
x = np.arange(1000)
y = pred[:1000]
####################
#Visuvalization
canvas = plt.figure()
rect = canvas.patch
rect.set_facecolor('white')
sp1 = canvas.add_subplot(1,1,1, axisbg='w')
sp1.plot(x, y, 'Red', linewidth=2)
sp1.plot(X, Y, 'Black', linewidth=1)

sp1.patch.set_facecolor('white') #------> background colour

sp1.tick_params(axis='x', colors='green') #---------> X-value colour
sp1.tick_params(axis='y', colors='red') #----------> y-value colour

sp1.spines['bottom'].set_color('black')        #Colour of spines 
sp1.spines['top'].set_visible('False')
sp1.spines['left'].set_color('black')
sp1.spines['right'].set_visible('False')

sp1.set_title('Impressions vs time Using Extra Classifier', color='red')
sp1.set_xlabel('Time,BLUE -Actual', color='red')      #Colour of title and lables 
sp1.set_ylabel('Impression', color='red')
plt.plot(X,Y)
plt.plot(x,y)

plt.tight_layout()
plt.show()

In [ ]:
len(pred)

In [ ]:
df_hp

In [10]:
hospital_dis= {}
for i in np.unique(df_hp['Hospital_ID'].values):
    hospital_dis[i] = {}
    index = [j for j,x in enumerate(df_hp['Hospital_ID'].values) if x == i]
    hospital_dis[i] = len(np.unique([df_hp['District_ID'].values[k]for k in index]))   

In [11]:
hospital_em = {}
for i in np.unique(df_hp['Hospital_ID'].values):
    hospital_em[i] = {}
    index = [j for j,x in enumerate(df_hp['Hospital_ID'].values) if x == i]
    hospital_em[i] = sum([df_hp['Hospital_employees'].values[k]for k in index])  
hospital_em['Hospital 1227'] = 100
hospital_em['Hospital 1234']  = 50
hospital_em['Hospital 1282']  = 50
hospital_em['Hospital 1293']  = 50
hospital_em['Hospital 174']  = 50
hospital_em['Hospital 408']  = 50

In [12]:
instrument_performance = {}
instrument_count = {}
s = 0
l = 0
for i in np.unique(df_hr['Instrument_ID'].values):
    for j in range(len(df_hr)):
        if df_hr['Instrument_ID'].values[j] == i:
            l = l+df_hr['Year Total'].values[j]
            s = s+1
    instrument_count[i] = s
    instrument_performance[i] = l

In [13]:
district_hos= {}
for i in np.unique(df_hr['District_ID'].values):
    district_hos[i] = {}
    index = [j for j,x in enumerate(df_hr['District_ID'].values) if x == i]
    district_hos[i] = len(np.unique([df_hr['Hospital_ID'].values[k]for k in index]))
district_hos['District 53'] = 50

In [14]:
district_em = {}
for i in np.unique(df_hp['District_ID'].values):
    district_em[i] = {}
    index = [j for j,x in enumerate(df_hp['District_ID'].values) if x == i]
    district_em[i] = sum([df_hp['Hospital_employees'].values[k]for k in index]) 
district_em['District 50'] = 100
district_em['District 51'] = 100
district_em['District 52'] = 100
district_em['District 47'] = 100

In [15]:
df_pr = df_pr[df_pr.District_ID != 'No District Available']
df_hr = df_hr[df_hr.District_ID != 'No District Available']

# Training 

In [17]:
#hospital_district = [hospital_dis[k]for k in df_pr['Hospital_ID'].values]
#hospital_empl = [hospital_em[k]for k in df_s['Hospital_ID'].values]
district_hosp = [district_hos[k]for k in df_pr['District_ID'].values]
district_emp = [district_em[k]for k in df_pr['District_ID'].values]
instrument_per = [instrument_performance[k]/instrument_count[k] for k in df_pr['Instrument_ID'].values]
#df_pr['hospital_district'] = hospital_district
df_pr['district_hosp'] = district_hosp
df_pr['district_emp'] = district_emp
df_pr['instrument_per'] = instrument_per

In [51]:
#hospital_district = [hospital_dis[k]for k in df_s['Hospital_ID'].values]
#hospital_empl = [hospital_em[k]for k in df_s['Hospital_ID'].values]
district_hosp = [district_hos[k]for k in df_s['District_ID'].values]
district_emp = [district_em[k]for k in df_s['District_ID'].values]
instrument_per = [instrument_performance[k]/instrument_count[k] for k in df_s['Instrument_ID'].values]
#df_s['hospital_district'] = hospital_district
df_s['district_hosp'] = district_hosp
df_s['district_emp'] = district_emp
df_s['instrument_per'] = instrument_per

In [52]:
#hospital_district = [hospital_dis[k]for k in df_hr['Hospital_ID'].values]
#hospital_empl = [hospital_em[k]for k in df_s['Hospital_ID'].values]
district_hosp = [district_hos[k]for k in df_hr['District_ID'].values]
district_emp = [district_em[k]for k in df_hr['District_ID'].values]
instrument_per = [instrument_performance[k]/instrument_count[k] for k in df_hr['Instrument_ID'].values]
#df_hr['hospital_district'] = hospital_district
df_hr['district_hosp'] = district_hosp
df_hr['district_emp'] = district_emp
df_hr['instrument_per'] = instrument_per

In [53]:

del df_hr['Month 1']
del df_hr['Month 2']  
del df_hr['Month 3']   
del df_hr['Month 4']   
del df_hr['Month 5'] 
del df_hr['Month 6']   
del df_hr['Month 7']   
del df_hr['Month 8']   
del df_hr['Month 9']   
del df_hr['Month 10']   
del df_hr['Month 11']   
del df_hr['Month 12']   
del df_hr['Region_ID']   

In [54]:
df_hr

,Hospital_ID,District_ID,Instrument_ID,Year Total,district_hosp,district_emp,instrument_per
0,Hospital 1,District 12,Instrument 2,103550,643,1823095,43537
1,Hospital 1,District 12,Instrument 3,2562,643,1823095,35777
2,Hospital 1,District 13,Instrument 1,187,1054,1881163,3183
3,Hospital 1,District 13,Instrument 2,40741,1054,1881163,43537
4,Hospital 1,District 13,Instrument 3,8549,1054,1881163,35777
5,Hospital 1,District 13,Instrument 4,1421,1054,1881163,33421
6,Hospital 1,District 13,Instrument 5,4820,1054,1881163,34052
7,Hospital 1,District 16,Instrument 2,10338,191,184457,43537
8,Hospital 1,District 18,Instrument 1,15337,631,767213,3183
9,Hospital 1,District 18,Instrument 2,212707,631,767213,43537


In [55]:
Y = df_hr['Year Total'].values
del df_hr['Year Total']   

In [56]:
del df_s['Buy_or_not']
del df_s['Revenue']

In [57]:
from sklearn.feature_extraction import DictVectorizer 
#ENCODING
X_train = df_hr.to_dict('records')
X_test = df_s.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te
#One Hot Encoding 
enc = DictVectorizer()
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_encoded_test =X_encoded_total[len(X_tr):]

In [64]:
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
reg = SVR()
reg.fit(X_encoded_train,Y )

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
pred = reg.predict(X_encoded_test)

In [60]:
df_sol = pandas.read_csv('Solution.csv')

In [61]:
df_sol['Revenue'] = pred

In [63]:

df_sol.to_csv('Solution.csv')

In [ ]:
 Y_pred_projected = reg.predict(X_encoded_test)

In [ ]:
Y_test = Y_Test
Y_pred = Y_pred_projected
MMSE = []
BMSE = []
for i in range(len(Y_test)):
        MMSE.append((Y_pred[i] - Y_test[i])**2)
        BMSE.append((Y_test[i] - np.mean(Y_test))**2)
mmse = sum(MMSE)/len(Y_pred)
bmse = sum(BMSE)/len(Y_pred)
s = (bmse- mmse)/bmse


In [ ]:
s